---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [184]:
import numpy as np 

K = 50                      # strike price
r = 0.02                    # risk-free rate per period
q = 0.02                    # dividend yield
sigma = 0.40                # volatility per year
T = 1                       # years to maturity
N = 40                      # number of time steps in tree

# times at which to calculate the boundary
times = T * np.array([0, 0.25, 0.5, 0.75, 0.9, 0.95, 0.98, 1])

# CALCULATING AMERICAN OPTION VALUES

To speed up the calculation of an American option value, we use the Black-Scholes formula at the penultimate date rather than the binomial tree to calculate the value of the option if not exercised at that date.  This is sensible, because there are no further opportunities to exercise after that date except at maturity, so the option is European if not exercised at the penultimate date.  

In [189]:
import numpy as np
from scipy.stats import norm

def BS(S, time_to_mat, kind):
    S = np.maximum(S, 1.0e-6)
    d1 = np.log(S / K) + (r - q + 0.5 * sigma ** 2) * time_to_mat
    d1 /= sigma * np.sqrt(time_to_mat)
    d2 = d1 - sigma * np.sqrt(time_to_mat)
    if kind == "call":
        return (
            np.exp(-q * time_to_mat) * S * norm.cdf(d1) - 
            np.exp(-r * time_to_mat) * K * norm.cdf(d2)
        )
    else:
        return (
            np.exp(-r * time_to_mat) * K * norm.cdf(-d2) - 
            np.exp(-q * time_to_mat) * S * norm.cdf(-d1)
        )
   

def American(S, time_to_mat, kind):
    
    intrinsic = lambda x: (x - K if kind == "call" else K - x)
    dt = time_to_mat / N
    up = np.exp(sigma * np.sqrt(dt))
    down = 1 / up
    prob = (np.exp((r - q) * dt) - down) / (up - down)
    discount = np.exp(-r * dt)

    # Black-Scholes at penultimate date
    x = S * up ** np.arange(N - 1, -N - 1, -2)
    v = np.maximum(intrinsic(x), BS(x, dt, kind))

    # step backward in the tree until date 1
    for n in range(N-2, 0, -1):
        x = S * up ** n
        v[0] = np.maximum(
            intrinsic(x), 
            discount * (prob * v[0] + (1 - prob) * v[1])
        )
        for i in range(1, n + 1):
            x *= down * down
            v[i] = np.maximum(
                intrinsic(x), 
                discount * (prob * v[i] + (1 - prob) * v[i + 1])
            )
    # calculate value at date 0 if not exercised (will compare to intrinsic value later)
    return discount * (prob * v[0] + (1 - prob) * v[1])

# CALCULATING THE EXERCISE BOUNDARY

We calculate at each time in a grid the price for the underlying asset at which the American option value just equals the intrinsic value.  This is the price at which it is optimal to exercise the option.  To be somewhat more precise, for calls we find
the maximum price and for puts we find the minimum price at which the American value is at least a penny above the intrinsic value.  The "penny" is a tolerance for numerical error.   

In [190]:
from scipy.optimize import minimize 

def Boundary(kind):
    intrinsic = lambda x: (max(x-K, 0) if kind == "call" else max(K-x, 0))
    bdys = []

    for t in times[:-1]:

        constraints = [{
            "type": "ineq",
            "fun": lambda x: American(x, T-t, kind) - intrinsic(x) - 0.01
        }]
        objective = lambda x: x if kind=="put" else -x

        b = minimize(objective, x0=K/100, method="SLSQP", constraints=constraints).x[0]
        bdys.append(b)

    bdys.append(K)
    return bdys


# CREATE FIGURES

In [191]:
import plotly.graph_objects as go 
import plotly.io as pio
colors = pio.templates["plotly_white"].layout.colorway

figs = []
for kind in ["call", "put"]:

    # add horizontal line at strike
    fig = go.Figure(
        go.Scatter(
            x=[0, T], 
            y=[K, K], 
            mode="lines", 
            line=dict(dash="dot", color=colors[1]),
            hovertemplate=None
        )
    )
  
    # call boundary when dividend yield>0
    if (kind=="call") and (q>0):
        b = Boundary(kind)

        # for adding shading
        maxvalue = np.max(b)
        fig.add_trace(
            go.Scatter(
                x=times,
                y=[maxvalue]*len(times),
                mode="lines",
                line_color=colors[0]
            )
        )

        # add boundary
        string = "boundary = $%{y:,.2f} at t = %{x:0.2f}<extra></extra>"
        fig.add_trace(
            go.Scatter(
                x=times,
                y=Boundary("call"),
                mode="lines",
                hovertemplate=string,
                line_color=colors[0],
                fill="tonexty",
            )
        )

    # put boundary
    elif kind=="put":
        fig.add_trace(
            go.Scatter(
                x=times,
                y=Boundary("put"),
                mode="lines",
                hovertemplate=string,
                line_color=colors[0],
                fill="tozeroy",
            )
        )

    # layout for call and put
    fig.update_layout(
        xaxis_title="Time (years)",
        yaxis_title="Underlying Price",
        xaxis_tickformat=",.1f",
        yaxis_tickformat=",.0f", 
        yaxis_tickprefix="$",
        yaxis_rangemode="tozero",
        template="plotly_white",
        showlegend=False
    )
    figs.append(fig)
        

    

# CALL FIGURE

In [192]:
figs[0].show()

# PUT FIGURE

In [193]:
figs[1].show()